In [26]:
import pandas as pd
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util
import json
import pyodbc

In [27]:
df_q=pd.read_excel("C://Users//DanukaDilshanRathnay//Desktop//AI-Driven-Job-Role-Fit-Prediction//code//Dataset//Similarity_with_EMP_data.xlsx")

In [28]:
server = 'DESKTOP-2DSGQFI'
driver = '{ODBC Driver 17 for SQL Server}'
default_database = 'ABC_Company'
connection_string = f"DRIVER={driver};SERVER={server};DATABASE={default_database};Trusted_Connection=yes"

conn = pyodbc.connect(connection_string, autocommit=True)
cursor = conn.cursor()
print("Connected to SQL Server")
def get_jd_data():
    query = "SELECT details FROM JD_Collection WHERE possition='Data Scientist'"
    return pd.read_sql(query, conn)

# Fetch Job Description data
df_jd = get_jd_data()
conn.close()

Connected to SQL Server


C:\Users\DanukaDilshanRathnay\AppData\Local\Temp\ipykernel_14540\3516608023.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


In [29]:
df1 = pd.read_excel('C://Users//DanukaDilshanRathnay//Desktop//AI-Driven-Job-Role-Fit-Prediction//code//Dataset//Employee8.xlsx')
df1=df1[df1['Department']=='Data and AI']
df1.head()

,EmployeeCode,FullName,Gender,Age,Department,JobCategory,ProficiencyLevel,Education Qualifications,Years of Experience in this Company,List of Technical Skills,...,Total Experience in Years,Number of Goal Assigned,Number of Goals Achieved,Final Score,Goals Score,Competency Score,Cultural Value Score,Additional Accomplishment Score,Potential Assessment Score,Trait Assessment Score
0,EMP9449,Tyler Pittman,Male,58,Data and AI,Data Engineer,Expert,"Bachelor of Applied Statistics, PhD in Statistics",10,"Spark, SQL, AWS",...,36,15,7,35.344631,46.666667,21.816859,75.686166,0.51,32.121590,50.741435
4,EMP9883,William Moore,Male,24,Data and AI,Data Scientist,Junior,"Bachelor of Engineering, Master of Data Analytics",2,"Pandas, Deep Learning, Python",...,2,2,0,35.000000,0.000000,17.464617,71.723773,5.74,0.000000,52.346808
5,EMP9479,Lindsey Patterson,Male,28,Data and AI,Data Engineer,Mid-Level,"Bachelor of Data Science, Master of AI",5,"AWS, Kafka, Spark",...,6,8,3,35.443860,37.500000,25.768876,55.723824,1.92,29.967015,30.604851
7,EMP9361,Joseph Olsen,Male,37,Data and AI,Data Engineer,Mid-Level,"Bachelor of Computer Science, Master of Data S...",14,"Hadoop, Kafka, SQL",...,15,15,12,42.225184,80.000000,40.971687,84.301638,9.29,60.239247,68.738249
8,EMP9041,Renee Hernandez,Male,36,Data and AI,Data Scientist,Mid-Level,"Bachelor of Engineering, PhD in AI & Automation",14,"Deep Learning, Python, Pandas",...,14,15,9,61.009778,60.000000,31.378751,62.786462,7.92,50.760558,50.995299


In [30]:
# Ensure job descriptions are extracted correctly
job_descriptions = df_jd['details'].tolist()
# df1 = pd.read_excel('C://Users//DanukaDilshanRathnay//Desktop//AI-Driven-Job-Role-Fit-Prediction//com//Employee6.xlsx')


columns_to_use = [
    'EmployeeCode', 'FullName', 'Gender', 'Age',"Department",
    'JobCategory', 'ProficiencyLevel', 'Education Qualifications',
    'Professional Qualifications', 'Total Experience in Years',
    'List of Technical Skills', 'List of Programming Skills',
    'List of Software Skills', 'List of Soft Skills', 'Projects Completed'
]
df1 = df1[columns_to_use]


df1.fillna("", inplace=True)


def format_employee_profile(row):
    profile_dict = {
        "Full Name": row['FullName'],
        #"Age": row['Age'],
        "Department": row['Department'],
        "Job Category": row['JobCategory'],
        "Proficiency Level": row['ProficiencyLevel'],
        "Education": row['Education Qualifications'],
        "Professional Certifications": row['Professional Qualifications'],
        "Total Experience (Years)": row['Total Experience in Years'],
        "Technical Skills": row['List of Technical Skills'],
        "Programming Skills": row['List of Programming Skills'],
        "Software Skills": row['List of Software Skills'],
        "Soft Skills": row['List of Soft Skills'],
        "Projects Completed": row['Projects Completed']
    }
    return json.dumps(profile_dict, ensure_ascii=False, indent=4)

# Apply formatting function to generate textual profiles
df1["EmployeeProfileText"] = df1.apply(format_employee_profile, axis=1)

In [31]:

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


employee_embeddings = model.encode(df1['EmployeeProfileText'].tolist(), convert_to_tensor=True)


similarity_scores = []
for emp_emb in employee_embeddings:
    scores = [util.cos_sim(emp_emb, model.encode(jd, convert_to_tensor=True)).item() for jd in job_descriptions]
    similarity_scores.append(max(scores))  


df1['JD match Score'] = similarity_scores
df1.to_excel("matchscore.xlsx",index=False)

In [32]:
df_jdmatch=df1[["EmployeeCode",'JD match Score','EmployeeProfileText']]
df_jdmatch.head()

,EmployeeCode,JD match Score,EmployeeProfileText
0,EMP9449,0.608949,"{\n ""Full Name"": ""Tyler Pittman"",\n ""Dep..."
4,EMP9883,0.551544,"{\n ""Full Name"": ""William Moore"",\n ""Dep..."
5,EMP9479,0.561577,"{\n ""Full Name"": ""Lindsey Patterson"",\n ..."
7,EMP9361,0.643902,"{\n ""Full Name"": ""Joseph Olsen"",\n ""Depa..."
8,EMP9041,0.593149,"{\n ""Full Name"": ""Renee Hernandez"",\n ""D..."


In [33]:
df_jdmatch.to_excel('C://Users//DanukaDilshanRathnay//Desktop//AI-Driven-Job-Role-Fit-Prediction//code//Dataset//Jd_score.xlsx',index=False)